### 1. Does data complexity/amount of zeros in data impact performance?

In [2]:
from mnist import *

In [3]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images  # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)

Instructions for updating:
Please use tf.data.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [18]:
train_data[0].shape[0] / 4

196.0

In [19]:
print(np.count_nonzero(train_data[0]))
# using only half of the image, approximately with the same amount of nonzero pixels
print(np.count_nonzero(train_data[0][196:3*196]))

168
145


Does the amount of nonzero pixels influence the neural networks speed?

In [49]:
train_data.shape

(55000, 784)

In [50]:
fake_data = np.zeros((55000, 784), dtype='float32')

In [51]:
train_labels.shape

(55000,)

In [52]:
fake_labels = np.zeros((55000), dtype='int32')

In [65]:
mnist_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn)

train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=32,
    num_epochs=None,
    shuffle=True)

train_input_fn_fake = tf.estimator.inputs.numpy_input_fn(
    x={"x": fake_data},
    y=fake_labels,
    batch_size=32,
    num_epochs=None,
    shuffle=True)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmplkbvudir', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f121b67fda0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [66]:
mnist_classifier.train(input_fn=train_input_fn, steps=300)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmplkbvudir/model.ckpt.
INFO:tensorflow:loss = 2.3090582, step = 1
INFO:tensorflow:global_step/sec: 19.9029
INFO:tensorflow:loss = 0.14983234, step = 101 (5.025 sec)
INFO:tensorflow:global_step/sec: 19.6196
INFO:tensorflow:loss = 0.17314973, step = 201 (5.097 sec)
INFO:tensorflow:Saving checkpoints for 300 into /tmp/tmplkbvudir/model.ckpt.
INFO:tensorflow:Loss for final step: 0.030321907.


In [55]:
mnist_classifier.train(input_fn=train_input_fn_fake, steps=300)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into mnist_convnet_model/model.ckpt.
INFO:tensorflow:loss = 2.3025851, step = 1
INFO:tensorflow:global_step/sec: 19.8262
INFO:tensorflow:loss = 2.3053055, step = 101 (5.045 sec)
INFO:tensorflow:global_step/sec: 19.9889
INFO:tensorflow:loss = 2.2988348, step = 201 (5.003 sec)
INFO:tensorflow:Saving checkpoints for 300 into mnist_convnet_model/model.ckpt.
INFO:tensorflow:Loss for final step: 2.2991505.


Apparently it is irrelevant how many zeros are in the data and whether the data makes sense. (both classifier take ~20 steps/sec)

### 2. Impact of x-value size on performance

In [82]:
def cnn_model_fn_big(features, labels, mode):
    """Model function for CNN."""
    # labels is just a tensor of size [batch_size]
    # features["x"] is actually just a flattened tensor that has to be reshaped
    input_layer = tf.reshape(features["x"], [-1, 56, 56, 1])

    # output tensor shape: [batch_size, 56, 56, 32]
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)

    # output tensor shape: [batch_size, 28, 28, 32]
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # output tensor shape: [batch_size, 28, 28, 64]
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)

    # output tensor shape: [batch_size, 14, 14, 64]
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    pool2_flat = tf.reshape(pool2, [-1, 14 * 14 * 64])

    # output tensor shape: [batch_size, 1024]
    dense = tf.layers.dense(
        inputs=pool2_flat, units=1024, activation=tf.nn.relu)

    dropout = tf.layers.dropout(
        inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # output tensor shape: [batch_size, 10]
    logits = tf.layers.dense(inputs=dropout, units=10)

    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer()
        train_op = optimizer.minimize(
            loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(
            mode=mode, loss=loss, train_op=train_op)

    elif mode == tf.estimator.ModeKeys.EVAL:
        eval_metric_ops = {
            "accuracy":
            tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])
        }
        return tf.estimator.EstimatorSpec(
            mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [79]:
classifier = tf.estimator.Estimator(model_fn=cnn_model_fn)
train_input_fn_small = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.zeros((55000, 784), dtype='float32')},
    y=np.zeros(55000, dtype='int32'),
    batch_size=32,
    num_epochs=None,
    shuffle=True)

classifier_big = tf.estimator.Estimator(model_fn=cnn_model_fn_big)
train_input_fn_big = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.zeros((55000, 56*56), dtype='float32')},
    y=np.zeros(55000, dtype='int32'),
    batch_size=32,
    num_epochs=None,
    shuffle=True)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmplecvuhb5', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1225e6a0f0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp3x5usbqz', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 10

In [80]:
classifier.train(input_fn=train_input_fn_small, steps=300)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmplecvuhb5/model.ckpt.
INFO:tensorflow:loss = 2.3025851, step = 1
INFO:tensorflow:global_step/sec: 19.6346
INFO:tensorflow:loss = 2.1251247, step = 101 (5.094 sec)
INFO:tensorflow:global_step/sec: 19.9105
INFO:tensorflow:loss = 1.953807, step = 201 (5.023 sec)
INFO:tensorflow:Saving checkpoints for 300 into /tmp/tmplecvuhb5/model.ckpt.
INFO:tensorflow:Loss for final step: 1.791132.


In [81]:
classifier_big.train(input_fn=train_input_fn_big, steps=300)

INFO:tensorflow:Calling model_fn.
raasrteinarsotarndeoiahnostrien
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmp3x5usbqz/model.ckpt.
INFO:tensorflow:loss = 2.3025851, step = 1
INFO:tensorflow:global_step/sec: 5.06731
INFO:tensorflow:loss = 2.1251247, step = 101 (19.735 sec)
INFO:tensorflow:global_step/sec: 5.03878
INFO:tensorflow:loss = 1.953807, step = 201 (19.846 sec)
INFO:tensorflow:Saving checkpoints for 300 into /tmp/tmp3x5usbqz/model.ckpt.
INFO:tensorflow:Loss for final step: 1.791132.


Apparently the size of the image is highly impactful on the training speed: making the input data 4 times as big makes the network 4 times as slow. (from 20 step/sec to 5 step/sec)